In [1]:
import pandas as pd
import numpy as np

# HURTLEX

In [2]:
hurtlex = pd.read_csv('hurtlex.txt', sep = '\t')
hurtlex

,id,pos,category,stereotype,lemma,level
0,ES3680,n,or,no,brassica campestris ssp. rapa,inclusive
1,ES241,n,or,no,sesera,inclusive
2,ES3699,n,re,no,privación ilegítima de la libertad,inclusive
3,ES573,n,dmc,yes,trolero,conservative
4,ES354,n,cds,no,lupanar,inclusive
...,...,...,...,...,...,...
5001,ES4210,n,qas,no,barraca,inclusive
5002,ES1923,n,cds,no,proxenetas,conservative
5003,ES3904,n,re,no,violador,conservative
5004,ES3822,n,dmc,yes,segundo evaluador,conservative


In [3]:
hurtlex[hurtlex.stereotype == 'yes']

,id,pos,category,stereotype,lemma,level
3,ES573,n,dmc,yes,trolero,conservative
6,ES2661,n,dmc,yes,tramposo,conservative
7,ES792,n,ddp,yes,zoofilia,conservative
12,ES379,a,dmc,yes,avaro,conservative
14,ES1902,n,dmc,yes,pasar-todo,inclusive
...,...,...,...,...,...,...
4998,ES2629,n,is,yes,pauperismo,inclusive
4999,ES1899,n,dmc,yes,libertinos,conservative
5000,ES4123,n,dmc,yes,ciruja,inclusive
5004,ES3822,n,dmc,yes,segundo evaluador,conservative


*Aqui lo estoy haciendo como listas, pero debieramos tenerlos como onehot encoders*

In [4]:
THlx = []
for i in range(len(hurtlex)):
    if len(hurtlex.lemma[i].split()) < 2:
        THlx.append((hurtlex.lemma[i], hurtlex.category[i]))
ExHlx_2 = []
for i in range(len(hurtlex)):
    if len(hurtlex.lemma[i].split()) == 2:
        ExHlx_2.append((hurtlex.lemma[i], hurtlex.category[i]))
ExHlx_3 = []
for i in range(len(hurtlex)):
    if len(hurtlex.lemma[i].split()) == 3:
        ExHlx_3.append((hurtlex.lemma[i], hurtlex.category[i]))

In [5]:
len(THlx), len(ExHlx_2), len(ExHlx_3)

(4501, 331, 108)

In [6]:
len(pd.DataFrame(THlx)[0].unique())

2935

# SHARE

In [7]:
with open('SHARE.txt', 'r', encoding = 'utf-8') as my_file:
    s = my_file.readlines()
TOf_S = []
ExOf_S = []
for i in s:
    if len(i.split()) < 2:
        TOf_S.append(i.replace('\n', ''))
    else:
        ExOf_S.append(i.replace('\n', ''))
TOf_S.append('bastardo')

In [8]:
TOf_S_not_in_Hlx = set(TOf_S) - set(pd.DataFrame(THlx)[0])

# Get emmbedings

In [9]:
from gensim.models import FastText

In [10]:
#!pip install fasttext
import fasttext
import fasttext.util

#fasttext.util.download_model('es', if_exists='ignore')  # Spanish
ft = fasttext.load_model('cc.es.300.bin')

In [ ]:
#from gensim.models.keyedvectors import KeyedVectors
#wordvectors_file_vec = '/home/posgrado/Documentos/Posgrado/Mineria/embeddings_spanish/SBW-vectors-300-min5.bin/sbw_vectors.bin'
#cantidad = 100000
#model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, binary=True)

In [11]:
def get_embedding(word):
    try:
        embedding = ft[word]
    except:
        embedding = np.zeros((300,))
        print('no existe la palabra ', word)
    return embedding.reshape(1, -1)

In [12]:
TOf_S_not_in_Hlx_embedding = [get_embedding(word) for word in TOf_S_not_in_Hlx]

In [14]:
len(TOf_S_not_in_Hlx_embedding)

5585

In [15]:
THlx_embedding = [get_embedding(word) for word in pd.DataFrame(THlx)[0]]

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
def get_cosine_similarity(vec, vec_all):
    cos_arr = [cosine_similarity(vec, vec_i) for vec_i in vec_all]
    return np.concatenate(tuple(cos_arr))
    

In [18]:
cosine_similarity_array = [get_cosine_similarity(emb, THlx_embedding) for emb in TOf_S_not_in_Hlx_embedding]

In [19]:
# indices de la palabra con mayor simulitud para cada TOf_S_not_in_Hlx en THlx
ind_max_sim = np.argmax(cosine_similarity_array, axis = 1)

In [21]:
ind_max_sim

array([[1106],
       [1505],
       [2340],
       ...,
       [2690],
       [1094],
       [3213]], dtype=int64)

In [59]:
#luego seria algo así para conseguir la palabra y la clasificación más cercana 
THlx[ind_max_sim[51][0]]

('fenículo', 'om')

In [25]:
TOf_S_not_in_Hlx

{'fondón',
 'submormal',
 'pollardo',
 'insoportable',
 'macán',
 'zorollo',
 'mamón',
 'perrocostra',
 'mamiluco',
 'superidiota',
 'matamierdas',
 'ordinaria',
 'egilipollas',
 'beocio',
 'pedoapestoso',
 'raspamonedas',
 'caranabo',
 'tragacuescos',
 'zarapastroso',
 'chihuahueño',
 'concho',
 'monjaplasta',
 'ansotermico',
 'soplapitos',
 'bergante',
 'pelleja',
 'babieco',
 'zampalimosna',
 'baciabarjas',
 'coñogordo',
 'chuplamela',
 'carinabo',
 'huraño',
 'repizcacristales',
 'calzacarros',
 'bardaliego',
 'giliputimaricojoneta',
 'endeble',
 'jilipiyas',
 'cheposo',
 'guriato',
 'patetismo',
 'caraguanabana',
 'zanguangon',
 'jalaguiles',
 'aminalado',
 'lamedor',
 'surnormal',
 'chupasangre',
 'microcefalico',
 'corrupto',
 'galapuchón',
 'soplaflautas',
 'berzotas',
 'piltrafas',
 'jodte',
 'milindri',
 'milindre',
 'mangurrino',
 'aleleado',
 'cangallo',
 'reprimido',
 'cometronchos',
 'carachinva',
 'lunático',
 'calamidá',
 'díscolo',
 'tinton',
 'bausan',
 'pollachanclas